In [17]:
import random
import numpy as np
import math
from scipy.stats import norm
import scipy.stats as stats

# 當v ~ uniform(0, 10), w ~ uniform(0, 20)

#### 當樣本數很大時，模擬擲硬幣1百萬次，估計真值



In [18]:
n, T, H = 0, 0, 0

curve = [i * math.pi / 2 + math.pi / 4 for i in range(15)]
curve = [0] + curve
curve = list(map(lambda x: x * 9.8, curve))

num_coll = []

while n < 1000000:
    v = random.uniform(0, 10)
    w = random.uniform(0, 20)
    num = v * w
    num_coll.append((v, w))

    for i in range(len(curve) - 1):
        if curve[i] <= num < curve[i + 1]:
            if i % 2 == 0:
                H += 1

            else:
                T += 1
    n += 1

print("Heads:", H)
print("Tails:", T)

theta_0 = H/n
print("當n很大時，發生正面機率(真值):",theta_0)

Heads: 522740
Tails: 477260
當n很大時，發生正面機率(真值): 0.52274


#### **jackknife**:  $\theta=( \bar x, s^2)$

In [19]:
n = 0

curve = [i * math.pi / 2 + math.pi / 4 for i in range(15)]
curve = [0] + curve
curve = list(map(lambda x: x * 9.8, curve))

num_coll = []
t_h_coll = []

while n < 1000:
    v = random.uniform(0, 10)
    w = random.uniform(0, 20)
    num = v * w
    num_coll.append((v, w))

    for i in range(len(curve) - 1):
        if curve[i] <= num < curve[i + 1]:
            if i % 2 == 0:
                t_h_coll.append(1)

            else:
                t_h_coll.append(0)


    n += 1

theta_hat = np.mean(t_h_coll)

jack_coll = []

for i in range(len(t_h_coll)):
    re_coll = t_h_coll.copy()
    re_coll.pop(i)
    jack_coll.append(np.mean(re_coll))


theta_dot_hat = np.mean(jack_coll)

bias_j = (n-1) * (theta_dot_hat - theta_hat)

se2_j  = (n-1)/n * np.sum((jack_coll - theta_dot_hat)**2)

mse_j = bias_j ** 2 + se2_j

print("jackknife's MSE:", mse_j)

jackknife's MSE: 0.00024928828828827706


####   **Bootstrap**

In [20]:
theta_hat = np.mean(t_h_coll)

boot_coll = []
B = 1000

for i in range(B):
    re_coll = t_h_coll.copy()
    x_star = []
    for i in range(n):
        x_star.append(np.random.choice(re_coll))
    boot_coll.append(np.mean(x_star))


theta_dot_hat = np.mean(boot_coll)

bias_b = theta_dot_hat - theta_hat

se2_b  = 1/(B - 1) * np.sum((boot_coll - theta_dot_hat)** 2)

mse_b = bias_b ** 2 + se2_b

print("bootstrap's MSE:", mse_b)

bootstrap's MSE: 0.0002659037854494498


#### 95% CI for Theta by The Simple Method

In [21]:
sort_boot_coll = sorted(boot_coll)

alpha = 0.05

simple_v1 = math.floor((B + 1) * alpha / 2)

simple_v2 = math.floor((B + 1) * (1 - alpha / 2))


simple_LCL =  2 * theta_hat -  sort_boot_coll[simple_v2]

simple_UCL =  2 * theta_hat -  sort_boot_coll[simple_v1]


print("95% CI for theta by the simple method is[", simple_LCL, ",", simple_UCL, "]" )

95% CI for theta by the simple method is[ 0.4990000000000001 , 0.5640000000000001 ]


95% CI for Theta by The Percentile Method

In [22]:
percentile_v1 = math.floor((B + 1) * alpha / 2)

percentile_v2 = math.floor((B + 1) * (1 - alpha / 2))


percentile_LCL =  2 * theta_hat -  sort_boot_coll[percentile_v2]

percentile_UCL =  2 * theta_hat -  sort_boot_coll[percentile_v1]


print("95% CI for theta by the simple method is[", percentile_LCL, ",", percentile_UCL, "]" )

95% CI for theta by the simple method is[ 0.4990000000000001 , 0.5640000000000001 ]


#### 95% CI for Theta by The BC

In [23]:
z_alpha = norm.ppf( 1- alpha / 2)

In [24]:
c = 0
for i in range(B):
  if boot_coll[i] <= theta_hat:
    c += 1

z0 = norm.ppf( c / B )

In [25]:
BC_v1 = norm.cdf(2 * z0 - z_alpha)
BC_v2 = norm.cdf(2 * z0 + z_alpha)

BC_LCL = boot_coll[math.floor((B + 1) * BC_v1)]
BC_UCL = boot_coll[math.floor((B + 1) * BC_v2)]
print("95% CI for theta by the BC is[", BC_LCL, ",", BC_UCL, "]" )

95% CI for theta by the BC is[ 0.53 , 0.533 ]


#### 95% CI for Theta by The BCa

In [26]:
sum_a = 0
sum_b = 0
for i in range(len(t_h_coll)):
  new = t_h_coll.copy()
  del new[i]
  sum_a += (theta_hat - np.mean(new))** 3
  sum_b += (theta_hat - np.mean(new))** 2

a_j = sum_a / (6 * ((sum_b)** (3/2)))

In [27]:
beta_1 = norm.cdf(z0 + ((z0 - z_alpha)/(1 - a_j * (z0 - z_alpha))))
beta_2 = norm.cdf(z0 + ((z0 + z_alpha)/(1 - a_j * (z0 + z_alpha))))

BCa_LCL = boot_coll[math.floor((B + 1) * beta_1)]
BCa_UCL = boot_coll[math.floor((B + 1) * beta_2)]
print("95% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )

95% CI for theta by the BCa is[ 0.53 , 0.533 ]


#### 檢驗其是否是否接近真值出現正面機率為theta_0

$H_0$: p = theta_0
$H_A$: p ≠ theta_0

使用t-test檢驗

$t_0 = \frac{\hat{\theta}-\theta_0}{\hat{\sigma}(\hat{\theta})}$

$t_b = \frac{\hat{\theta_b}-\theta_0}{\hat{\sigma}(\hat{\theta_b})}$

計算 tb >= t0 個數除以B，以估計p-value


In [28]:
theta_hat = np.mean(t_h_coll)
s = np.std(t_h_coll)
t0 = abs((theta_hat - theta_0)/s)

b_s = np.std(boot_coll)

t = ([(x - theta_0) / b_s for x in boot_coll])

p_value = sum(t >= t0) / B
if p_value < 0.05:
  print("p-value:",p_value,"小於0.05，有足夠證據顯示拒絕H0,亦即此銅板不公正")
else:
  print("p-value:",p_value,"大於等於0.05，沒有足夠證據顯示拒絕H0")

p-value: 0.693 大於等於0.05，沒有足夠證據顯示拒絕H0


Bootstrap Tests by BCa

In [29]:
print("95% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )
if theta_0<= BCa_UCL and theta_0>= BCa_LCL:
    print("theta_0 is in the 95% CI for theta by the BCa")
    print("p-value:",alpha)
else:
    # 擴大信賴區間(alpha值)計算何時theta_0在裡面，此時alpha就是欲求的p-value
    new_alpha = alpha
    while new_alpha < 1:
      z_alpha = norm.ppf( new_alpha / 2)
      beta_1 = norm.cdf(z0 + ((z0 - z_alpha)/(1 - a_j * (z0 - z_alpha))))
      beta_2 = norm.cdf(z0 + ((z0 + z_alpha)/(1 - a_j * (z0 + z_alpha))))
      BCa_LCL = boot_coll[math.floor((B + 1) * beta_1)]
      BCa_UCL = boot_coll[math.floor((B + 1) * beta_2)]
      if theta_0 >= BCa_LCL and theta_0 <= BCa_UCL:
        break
      new_alpha += 0.0001
    print("p-value:",new_alpha)
    print("{:.3f}".format((1-round(new_alpha,4))*100),"% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )

95% CI for theta by the BCa is[ 0.53 , 0.533 ]
p-value: 0.055000000000000146
94.500 % CI for theta by the BCa is[ 0.505 , 0.536 ]


p值於0.05，表沒有足夠證據顯示正面的機率不是theta_0

# 當v ~ truncnorm(0, 10), w ~ truncnorm(0, 20)

#### 當樣本數很大時, 模擬擲硬幣1百萬次，估計真值

In [30]:
n, T, H = 0, 0, 0

mu_v, sigma_v = 5, 1
lower_v, upper_v = 0, 10

mu_w, sigma_w = 10, 1
lower_w, upper_w = 0, 20


X = stats.truncnorm((lower_v - mu_v)/sigma_v, (upper_v - mu_v)/sigma_v, loc=mu_v, scale=sigma_v)
Y = stats.truncnorm((lower_w - mu_w)/sigma_w, (upper_w - mu_w)/sigma_w, loc=mu_w, scale=sigma_w)

curve = [i * np.pi / 2 + np.pi / 4 for i in range(15)]
curve = [0] + curve
curve = list(map(lambda x: x * 9.8, curve))

num_coll = []
while n < 1000000:
    v = X.rvs(1)
    w = Y.rvs(1)
    num = v * w
    num_coll.append((v, w))
    for i in range(len(curve) - 1):
        if curve[i] <= num < curve[i + 1]:
            if i % 2 == 0:
                H += 1
            else:
                T += 1

    n += 1

print("Heads:", H)
print("Tails:", T)
theta_0  = H/n
print("當n很大時，發生正面機率(真值):",theta_0)

Heads: 452913
Tails: 547087
當n很大時，發生正面機率(真值): 0.452913


#### **jackknife**:  $\theta=( \bar x, s^2)$

In [31]:
n = 0

curve = [i * math.pi / 2 + math.pi / 4 for i in range(15)]
curve = [0] + curve
curve = list(map(lambda x: x * 9.8, curve))

mu_v, sigma_v = 5, 1
lower_v, upper_v = 0, 10

mu_w, sigma_w = 10, 1
lower_w, upper_w = 0, 20

X = stats.truncnorm((lower_v - mu_v)/sigma_v, (upper_v - mu_v)/sigma_v, loc=mu_v, scale=sigma_v)
Y = stats.truncnorm((lower_w - mu_w)/sigma_w, (upper_w - mu_w)/sigma_w, loc=mu_w, scale=sigma_w)
num_coll = []
t_h_coll = []

while n < 1000:
    v = X.rvs(1)
    w = Y.rvs(1)
    num = v * w
    num_coll.append((v, w))

    for i in range(len(curve) - 1):
        if curve[i] <= num < curve[i + 1]:
            if i % 2 == 0:
                t_h_coll.append(1)

            else:
                t_h_coll.append(0)


    n += 1

theta_hat = np.mean(t_h_coll)

jack_coll = []

for i in range(len(t_h_coll)):
    re_coll = t_h_coll.copy()
    re_coll.pop(i)
    jack_coll.append(np.mean(re_coll))


theta_dot_hat = np.mean(jack_coll)

bias_j = (n-1) * (theta_dot_hat - theta_hat)

se2_j  = (n-1)/n * np.sum((jack_coll - theta_dot_hat)**2)

mse_j = bias_j ** 2 + se2_j

print("jackknife's MSE:", mse_j)

jackknife's MSE: 0.00024774774774776403


#### **Bootstrap**

In [32]:
theta_hat = np.mean(t_h_coll)
B = 1000
boot_coll = []
for i in range(B):
    re_coll = t_h_coll.copy()
    x_star = []
    for i in range(n):
        x_star.append(np.random.choice(re_coll))
    boot_coll.append(np.mean(x_star))


theta_dot_hat = np.mean(boot_coll)

bias_b = theta_dot_hat - theta_hat

se2_b  = 1/(B - 1) * np.sum((boot_coll - theta_dot_hat)** 2)

mse_b = bias_b ** 2 + se2_b

print("bootstrap's MSE:", mse_b)

bootstrap's MSE: 0.0002403333322432433


#### 95% CI for Theta by The Simple Method

In [33]:
sort_boot_coll = sorted(boot_coll)

alpha = 0.05

simple_v1 = math.floor((B + 1) * alpha / 2)

simple_v2 = math.floor((B + 1) * (1 - alpha / 2))


simple_LCL =  2 * theta_hat -  sort_boot_coll[simple_v2]

simple_UCL =  2 * theta_hat -  sort_boot_coll[simple_v1]


print("95% CI for theta by the simple method is[", simple_LCL, ",", simple_UCL, "]" )

95% CI for theta by the simple method is[ 0.42000000000000004 , 0.47900000000000004 ]


#### 95% CI for Theta by The Percentile Method

In [34]:
percentile_v1 = math.floor((B + 1) * alpha / 2)

percentile_v2 = math.floor((B + 1) * (1 - alpha / 2))


percentile_LCL =  2 * theta_hat -  sort_boot_coll[percentile_v2]

percentile_UCL =  2 * theta_hat -  sort_boot_coll[percentile_v1]


print("95% CI for theta by the simple method is[", percentile_LCL, ",", percentile_UCL, "]" )

95% CI for theta by the simple method is[ 0.42000000000000004 , 0.47900000000000004 ]


#### 95% CI for Theta by The BC

In [35]:
z_alpha = norm.ppf( 1 - alpha / 2)

In [36]:
c = 0
for i in range(B):
  if boot_coll[i] <= np.mean(t_h_coll):
    c += 1

z0 = norm.ppf( c / B )

In [37]:
BC_v1 = norm.cdf(2 * z0 - z_alpha)
BC_v2 = norm.cdf(2 * z0 + z_alpha)

BC_LCL = boot_coll[math.floor((B+1)*BC_v1)]
BC_UCL = boot_coll[math.floor((B+1)*BC_v2)]
print("95% CI for theta by the BC is[", BC_LCL, ",", BC_UCL, "]" )

95% CI for theta by the BC is[ 0.447 , 0.452 ]


#### 95% CI for Theta by The BCa

In [38]:
sum_a = 0
sum_b = 0
for i in range(len(t_h_coll)):
  new = jack_coll.copy()
  del new[i]
  sum_a += (np.mean(jack_coll) - np.mean(new))** 3
  sum_b += (np.mean(jack_coll) - np.mean(new))** 2

a_j = sum_a / (6 * ((sum_b)** (3/2)))

In [39]:
beta_1 = norm.cdf(z0 + (z0 - z_alpha)/(1 - a_j * (z0 - z_alpha)))
beta_2 = norm.cdf(z0 + (z0 + z_alpha)/(1 - a_j * (z0 + z_alpha)))

BCa_LCL = boot_coll[math.floor((B+1)*beta_1)]
BCa_UCL = boot_coll[math.floor((B+1)*beta_2)]
print("95% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )

95% CI for theta by the BCa is[ 0.447 , 0.445 ]


#### 檢驗其是否是否接近真值出現正面機率為theta_0

$H_0$: p = theta_0
$H_A$: p ≠ theta_0

使用t-test檢驗

$t_0 = \frac{\hat{\theta}-\theta_0}{\hat{\sigma}(\hat{\theta})}$

$t_b = \frac{\hat{\theta_b}-\theta_0}{\hat{\sigma}(\hat{\theta_b})}$

計算 tb >= t0 個數除以B，以估計p-value


In [40]:
theta_hat = np.mean(t_h_coll)
s = np.std(t_h_coll)
t0 = abs((theta_hat - theta_0)/s)

b_s = np.std(boot_coll)

t = ([(x - theta_0) / b_s for x in boot_coll])

p_value = sum(t >= t0) / B
if p_value < 0.05:
  print("p-value:",p_value,"小於0.05，有足夠證據顯示拒絕H0,亦即此銅板不公正")
else:
  print("p-value:",p_value,"大於等於0.05，沒有足夠證據顯示拒絕H0")

p-value: 0.411 大於等於0.05，沒有足夠證據顯示拒絕H0


Bootstrap Tests by BCa

In [41]:
print("95% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )
if theta_0<= BCa_UCL and theta_0>= BCa_LCL:
    print("theta_0 is in the 95% CI for theta by the BCa")
    print("p-value:",alpha)
else:
    # 擴大信賴區間(alpha值)計算何時theta_0在裡面，此時alpha就是欲求的p-value
    new_alpha = alpha
    while new_alpha < 1:
      z_alpha = norm.ppf( new_alpha / 2)
      beta_1 = norm.cdf(z0 + ((z0 - z_alpha)/(1 - a_j * (z0 - z_alpha))))
      beta_2 = norm.cdf(z0 + ((z0 + z_alpha)/(1 - a_j * (z0 + z_alpha))))
      BCa_LCL = boot_coll[math.floor((B + 1) * beta_1)]
      BCa_UCL = boot_coll[math.floor((B + 1) * beta_2)]
      if theta_0 >= BCa_LCL and theta_0 <= BCa_UCL:
        break
      new_alpha += 0.0001
    print("p-value:",new_alpha)
    print("{:.2f}".format((1-round(new_alpha,4))*100),"% CI for theta by the BCa is[", BCa_LCL, ",", BCa_UCL, "]" )

95% CI for theta by the BCa is[ 0.447 , 0.445 ]
p-value: 0.06950000000000056
93.05 % CI for theta by the BCa is[ 0.426 , 0.481 ]


p值於0.05，表沒有足夠證據顯示正面的機率不是theta_0